<a href="https://colab.research.google.com/github/vai-ya/Object-Detection/blob/main/objectdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Project- Object Detection Using OpenCV
# Install required packages
!pip install ultralytics opencv-python-headless

import cv2
from ultralytics import YOLO
import numpy as np
from IPython.display import display, Javascript, HTML
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import time

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Using nano version for better performance

# Create output area
display(HTML('<div id="webcam-container" style="display: flex; justify-content: center;"></div>'))
display(HTML('<div id="detection-container" style="display: flex; justify-content: center;"></div>'))

# JavaScript to capture webcam and send frames
js = Javascript('''
async function setupWebcam() {
  const webcamContainer = document.getElementById('webcam-container');
  const detectionContainer = document.getElementById('detection-container');

  // Create video element for webcam
  const video = document.createElement('video');
  video.style.display = 'block';
  video.width = 640;
  video.height = 480;

  // Create canvas for capturing frames
  const canvas = document.createElement('canvas');
  canvas.width = 640;
  canvas.height = 480;
  const ctx = canvas.getContext('2d');

  webcamContainer.appendChild(video);

  // Start video stream
  const stream = await navigator.mediaDevices.getUserMedia({video: true});
  video.srcObject = stream;
  await video.play();

  // Function to capture frame and send to Python
  async function captureFrame() {
    ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
    const data = canvas.toDataURL('image/jpeg', 0.8);
    google.colab.kernel.invokeFunction('notebook.processFrame', [data], {});
    setTimeout(captureFrame, 100);
  }

  captureFrame();

  return stream;
}
''')

# Display JavaScript
display(js)

# Function to process frames (will be called from JavaScript)
def process_frame(data):
    # Convert base64 image to OpenCV format
    binary = b64decode(data.split(',')[1])
    image = np.frombuffer(binary, dtype=np.uint8)
    frame = cv2.imdecode(image, cv2.IMREAD_COLOR)

    # Run YOLOv8 inference
    results = model(frame)

    # Visualize results
    annotated_frame = results[0].plot()

    # Convert back to base64 for display
    _, encoded_image = cv2.imencode('.jpg', annotated_frame)
    display(HTML(f'''
      <img src="data:image/jpg;base64,{b64encode(encoded_image).decode()}" width="640"/>
    '''), display_id='detection-display')

# Register the Python callback
from google.colab import output
output.register_callback('notebook.processFrame', process_frame)

# Start the webcam
try:
    stream = eval_js('setupWebcam()')
    print("Webcam started. Press Ctrl+C to stop.")

    # Keep the Python process running
    while True:
        time.sleep(1)

except KeyboardInterrupt:
    # Stop the stream
    display(Javascript('''
      if (typeof stream !== 'undefined') {
        stream.getTracks().forEach(track => track.stop());
      }
    '''))
    print("Webcam stopped")
except Exception as e:
    print(f"Error: {str(e)}")
    display(Javascript('''
      if (typeof stream !== 'undefined') {
        stream.getTracks().forEach(track => track.stop());
      }
    '''))